In [1]:
import tensorflow as tf
from keras import datasets

In [2]:
# x: [60k, 28, 28]
# y: [60k]
(x, y), _ = datasets.mnist.load_data()

x = tf.convert_to_tensor(x, dtype=tf.float32) / 255.0
y = tf.convert_to_tensor(y, dtype=tf.int32)

print(x.shape, y.shape, x.dtype, y.dtype)
print(tf.reduce_min(x), tf.reduce_max(x))
print(tf.reduce_min(y), tf.reduce_max(y))

(60000, 28, 28) (60000,) <dtype: 'float32'> <dtype: 'int32'>
tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(0, shape=(), dtype=int32) tf.Tensor(9, shape=(), dtype=int32)


In [3]:
train_db = tf.data.Dataset.from_tensor_slices((x, y)).batch(128).repeat(10)

train_iter = iter(train_db)

sample = next(train_iter)
print("batch:", sample[0].shape, sample[1].shape)

batch: (128, 28, 28) (128,)


In [4]:
w1 = tf.Variable(tf.random.truncated_normal([784, 256], stddev=0.1))
b1 = tf.Variable(tf.zeros([256]))

w2 = tf.Variable(tf.random.truncated_normal([256, 128], stddev=0.1))
b2 = tf.Variable(tf.zeros([128]))

w3 = tf.Variable(tf.random.truncated_normal([128, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))
lr = 0.001

In [5]:


for step, (x, y) in enumerate(train_db):
    # x: [128, 28, 28]
    # y: [128]

    # [b, 28, 28] => [b, 28 * 28]
    x = tf.reshape(x, [-1, 28 * 28])

    with tf.GradientTape() as tape:
        # x: [b, 28 * 28]
        # h1 = x@w1 + b1
        # [b, 784] @ [784, 256] + [256] => [b, 256] + [256] => [b, 256] + [b, 256]
        h1 = x @ w1 + tf.broadcast_to(b1, [x.shape[0], 256])
        h1 = tf.nn.relu(h1)
        # [b, 256] => [b, 128]
        h2 = h1 @ w2 + b2
        h2 = tf.nn.relu(h2)
        out = h2 @ w3 + b3
        # compute loss
        # out: [b, 10]
        # y: [b] => [b, 10]
        y_onehot = tf.one_hot(y, depth=10)

        # mse = mean(sum(y-out)^2)
        # [b, 10]
        loss = tf.square(y_onehot - out)
        loss = tf.reduce_mean(loss)
        # loss = tf.reduce_mean(loss)

    grads = tape.gradient(loss, [w1, b1, w2, b2, w3, b3])
    w1.assign_sub(lr * grads[0])
    b1.assign_sub(lr * grads[1])
    w2.assign_sub(lr * grads[2])
    b2.assign_sub(lr * grads[3])
    w3.assign_sub(lr * grads[4])
    b3.assign_sub(lr * grads[5])
    if step % 100 == 0:
        print(step, "loss:", float(loss))


0 loss: 0.46193838119506836
100 loss: 0.2212197482585907
200 loss: 0.19767668843269348
300 loss: 0.19848474860191345
400 loss: 0.15623334050178528
500 loss: 0.1642087697982788
600 loss: 0.15863755345344543
700 loss: 0.16028380393981934
800 loss: 0.16142795979976654
900 loss: 0.13589081168174744
1000 loss: 0.14197833836078644
1100 loss: 0.14610497653484344
1200 loss: 0.1381656527519226
1300 loss: 0.13717544078826904
1400 loss: 0.11375860124826431
1500 loss: 0.1206444725394249
1600 loss: 0.12496642768383026
1700 loss: 0.12156470119953156
1800 loss: 0.12355168163776398
1900 loss: 0.10537280142307281
2000 loss: 0.11864757537841797
2100 loss: 0.10711614042520523
2200 loss: 0.10748650878667831
2300 loss: 0.10843000560998917
2400 loss: 0.10549235343933105
2500 loss: 0.10549177974462509
2600 loss: 0.10727670043706894
2700 loss: 0.11270985752344131
2800 loss: 0.10165496915578842
2900 loss: 0.09539054334163666
3000 loss: 0.10339958965778351
3100 loss: 0.09295061975717545
3200 loss: 0.10968729108

In [4]:
!wsl pwd

/mnt/d/project/python/machine_learning
